In [1]:
from datetime import datetime
from  os import name, path
from io import StringIO
from time import sleep
import json

import utils
from pipe import sort
import pprint 
from AvailableBalances import AvailableBalance
from Ledgers import Ledgers
import Orders
import Trades  

In [3]:
av = AvailableBalance()
bal = av.get_avalailable_balances()
pprint.pprint(bal) 

OrderedDict([('AAVE', Decimal('0E-10')),
             ('ADA', Decimal('1199.49426473')),
             ('BAND', Decimal('0E-10')),
             ('CRV', Decimal('0E-10')),
             ('DOT', Decimal('0E-10')),
             ('EOS', Decimal('0E-10')),
             ('EUR.HOLD', Decimal('0.0000')),
             ('EWT', Decimal('0E-10')),
             ('FIL', Decimal('0E-10')),
             ('GNO', Decimal('0E-10')),
             ('GRT', Decimal('0E-10')),
             ('ICX', Decimal('0E-10')),
             ('KAVA', Decimal('0E-8')),
             ('KSM', Decimal('0E-10')),
             ('MATIC', Decimal('2.8903747200')),
             ('OCEAN', Decimal('1.00E-8')),
             ('OXT', Decimal('0E-10')),
             ('PAXG', Decimal('0E-10')),
             ('QTUM', Decimal('0E-10')),
             ('RARI', Decimal('0E-10')),
             ('SAND', Decimal('0E-10')),
             ('STORJ', Decimal('0E-10')),
             ('TRX', Decimal('8.89447921')),
             ('USDC', Decimal('0E-8')),
